# Student Loan Risk with Deep Learning

In [50]:
# Imports
import pandas as pd
import tensorflow as tf
import keras
import os
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [51]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [52]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [53]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

In [54]:
# Check values per feature
loans_df.count()

payment_history           1599
location_parameter        1599
stem_degree_score         1599
gpa_ranking               1599
alumni_success            1599
study_major_code          1599
time_to_completion        1599
finance_workshop_score    1599
cohort_ranking            1599
total_loan_score          1599
financial_aid_score       1599
credit_ranking            1599
dtype: int64

In [55]:
# Check to see # of NaaN entries per feature
loans_df.isna().sum()/len(loans_df) # No null values in the DF

payment_history           0.0
location_parameter        0.0
stem_degree_score         0.0
gpa_ranking               0.0
alumni_success            0.0
study_major_code          0.0
time_to_completion        0.0
finance_workshop_score    0.0
cohort_ranking            0.0
total_loan_score          0.0
financial_aid_score       0.0
credit_ranking            0.0
dtype: float64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [56]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]

# Display a sample of y
y.head()

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [57]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.drop("credit_ranking", axis=1)

# Review the features DataFrame
X.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [58]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [59]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
scaler.fit(X_train)

# Fit the scaler to the features training dataset
X_test_scaled = scaler.transform(X_test)

---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [60]:
# Define the the number of inputs (features) to the model
input_features = X_train.shape[1] 
input_layer = layers.Input(shape=(input_features,))

# Review the number of features
print(input_features)

11


In [61]:
# Define the number of hidden nodes for the first hidden layer
hidden_nodes_layer1 = 6

# Define the number of hidden nodes for the second hidden layer
hidden_nodes_layer2 = 3

# Define the number of neurons in the output layer
output_neurons = 1

In [62]:
# Create a sequential model
model = keras.Sequential()

# Add the input layer
model.add(input_layer)

# Add the first hidden layer with ReLU activation
model.add(layers.Dense(hidden_nodes_layer1, activation='relu'))

# Add the second hidden layer with ReLU activation
model.add(layers.Dense(hidden_nodes_layer2, activation='relu'))

# Add the output layer (assuming binary classification)
model.add(layers.Dense(output_neurons, activation='sigmoid'))  # Use 'softmax' for multi-class classification


In [63]:
# Display the Sequential model summary
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │            21 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 97 (388.00 B)

 Trainable params: 97 (388.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [64]:
# Compile the Sequential model
model.compile(optimizer='adam',  # Adam optimizer
              loss='binary_crossentropy',  # Binary crossentropy loss function
              metrics=['accuracy'])  # Accuracy as evaluation metric

In [65]:
# Fit the model using 50 epochs and the training data
epochs = 50  # Set the number of epochs
batch_size = 13  # Set the batch size

history = model.fit(X_train, y_train,  # Training data
                    epochs=epochs, 
                    batch_size=batch_size)

Epoch 1/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5449 - loss: 3.4254
Epoch 2/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5419 - loss: 1.7120
Epoch 3/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.4331 - loss: 0.9927
Epoch 4/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.3807 - loss: 0.7733
Epoch 5/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5760 - loss: 0.6753
Epoch 6/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6501 - loss: 0.6526
Epoch 7/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6143 - loss: 0.6625
Epoch 8/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6382 - loss: 0.6513
Epoch 9/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6230 - loss: 0.6591
Epoch 10/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6599 - loss: 0.6455
Epoch 11/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6321 - loss: 0.6616
Epoch 12/50
93/93 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5957 - lo

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [66]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=0)

# Display the model loss and accuracy results
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Loss: 0.6290
Test Accuracy: 0.6375


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [67]:
# Define the directory where you want to save the model
model_directory = 'Resources'  # Use raw string to avoid escape character issues

# Ensure the directory exists
os.makedirs(model_directory, exist_ok=True)

# Define the full path for the model file
model_file_path = os.path.join(model_directory, 'student_loans.keras')

# Export your model to a keras file
model.save(model_file_path)
print(f"Model saved to: {model_file_path}")


Model saved to: Resources\student_loans.keras


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [68]:
# Set the model's file path
model_directory = 'Resources'
# Define the full path for the model file
model_file_path = os.path.join(model_directory, 'student_loans.keras')

# Load the model to a new object
loaded_model = load_model(model_file_path)

# Print the summary of the loaded model to confirm it has been loaded correctly
loaded_model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 6)              │            72 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 3)              │            21 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │             4 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 293 (1.15 KB)

 Trainable params: 97 (388.00 B)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 196 (788.00 B)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [69]:
# Make predictions with the test data
predictions = loaded_model.predict(X_test)

# Display a sample of the predictions
sample_size = 5  # Define the number of samples to display
print("Sample Predictions (Probabilities):")
print(predictions[:sample_size])  # Print the first few probability predictions

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
Sample Predictions (Probabilities):
[[0.52532184]
 [0.32248732]
 [0.6972574 ]
 [0.5700133 ]
 [0.68260217]]


In [70]:
# Save the predictions to a DataFrame and round the predictions to binary results
# Convert the probabilities to binary class labels
predicted_classes = (predictions > 0.5).astype(int)

# Create a DataFrame to hold the predictions
predictions_df = pd.DataFrame({
    'Predicted Probabilities': predictions.flatten(),  # Flatten to 1D array
    'Predicted Classes': predicted_classes.flatten()  # Flatten to 1D array
})

# Round the predictions to binary results (if needed, this is optional since we already converted)
predictions_df['Rounded Predictions'] = predictions_df['Predicted Probabilities'].round()

# Display the DataFrame
print(predictions_df)  # Display the first few rows of the DataFrame

     Predicted Probabilities  Predicted Classes  Rounded Predictions
0                   0.525322                  1                  1.0
1                   0.322487                  0                  0.0
2                   0.697257                  1                  1.0
3                   0.570013                  1                  1.0
4                   0.682602                  1                  1.0
..                       ...                ...                  ...
395                 0.678527                  1                  1.0
396                 0.627119                  1                  1.0
397                 0.659158                  1                  1.0
398                 0.322487                  0                  0.0
399                 0.842858                  1                  1.0

[400 rows x 3 columns]


### Step 4: Display a classification report with the y test data and predictions

In [71]:
# Print the classification report with the y test data and predictions
print("Classification Report:")
print(classification_report(y_test, predicted_classes))

Classification Report:
              precision    recall  f1-score   support

           0       0.66      0.46      0.55       188
           1       0.62      0.79      0.70       212

    accuracy                           0.64       400
   macro avg       0.64      0.63      0.62       400
weighted avg       0.64      0.64      0.63       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**
Student's demographics, credit history, income, ... data, to help with establish student's financial history, needs, and eligibility.
Types of loans, to give greater flexiblity to match student's needs with options.
Purpose of the loan, to align loan options to reasons.  And assess equity, amount, and other loan conditions.
Market data to provide current assessment and update recommendations engine.


**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**
Content-based filtering to help match students to personalized loan options.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**
Data Privacy and security.
The changing nature of loan products.  Must peg against market and update based on macro, micro economic trends as well as changes in loan product, rates, etc.  Without perpetual, current data both the student and the loan products can be out of step with market needs.
